## Acesso ao Integra via Aida

### Importa modulos

In [23]:
import adal
import json
import requests
import pandas as pd
from pathlib import Path
import numpy as np
import re
from datetime import datetime

#### Ignorando as mensagens de aviso ("Deprecation Warning") no python

In [24]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

#### Scope of Supply

In [25]:
# Definição do dataframe do pacote para WATER INJECTION PUMPS AND SEA WATER BOOSTER PUMPS

data_wip = {
'TAG': ['B-1251001A/C', 'M-B-1251001A/C', 'B-1251002A/C', 'M-B-1251002A/C', 'B-1251003A/B', 'M-B-1251003A/B', 'VV-B-1251002A/C', 'PN-B-1251002A/C'],
'QTY': ['3 x 50%', '3 set', '3 x 50%', '3 set', '2 x 100%', '2 set', '3 set', '3 set'],
'DESCRIPTION': ['INJECTION WATER BOOSTER PUMP', 'INJECTION WATER BOOSTER PUMP - MOTOR', 'INJECTION WATER MAIN PUMP', 'INJECTION WATER MAIN PUMP - MOTOR', 'SEAWATER BOOSTER PUMP', 'SEAWATER BOOSTER PUMP - MOTOR', 'HVSD FOR MAIN INJECTION WATER PUMP', 'MAIN INJECTION WATER PUMP CONTROL PANEL'],
'LOCATION': ['M-11', 'M-11', 'M-11', 'M-11', 'M-14', 'M-14', 'M-11', 'M-11'],
'AREA_CLASSIFICATION': ['Outdoor, Safe Area', 'Outdoor, Safe Area', 'Outdoor, Safe Area', 'Outdoor, Safe Area', 'Outdoor, Zone 2, T3, Group IIA', 'Outdoor, Zone 2, T3, Group IIA', 'Indoor, Safe Area', 'Indoor, Safe Area'],
'PKG_DESCRIPTION': ['WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS', 'WATER INJECTION PUMPS']
}

df_wip = pd.DataFrame(data_wip)
print('\n Scope of Supply for WATER INJECTION PUMPS AND SEA WATER BOOSTER PUMPS')
print(df_wip)

# Definição do dataframe do pacote para WATER LIFT PUMPS (B-5111001A/D AND B-5111002)

data_lift_pump = {
'TAG': ['B-5111001A/D', 'M-B-5111001A/D', 'B-5111002', 'M-B-5111002', 'PN-B-5111001A/D', 'PN-B-5111002-01', 'PN-B-5111002-02', 'UB-B-5111001A/D', 'UB-B-5111002'],
'QTY': ['4 x 33%', '4 x 33%', '1 x 100%', '1 x 100%', '4', '1', '1', '4', '1'],
'DESCRIPTION': ['SEA WATER LIFT PUMP', 'ELECTRIC MOTOR FOR SEA WATER LIFT PUMP', 'START-UP SEA WATER LIFT PUMP', 'ELECTRIC MOTOR FOR START-UP SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP CONTROL PANEL', 'START-UP SEA WATER LIFT PUMP CONTROL PANEL', 'START-UP SEA WATER LIFT PUMP POWER PANEL', 'FLUID CIRCULATION UNIT FOR SEA WATER LIFT PUMPS', 'FLUID CIRCULATION UNIT FOR START-UP SEA WATER LIFT PUMP'],
'LOCATION': ['MAIN DECK', 'MAIN DECK', 'MAIN DECK', 'MAIN DECK', 'M-17', 'M-17', 'ENGINE ROOM', 'MAIN DECK', 'MAIN DECK'],
'AREA_CLASSIFICATION': ['SAFE AREA', 'SAFE AREA', 'SAFE AREA', 'SAFE AREA', 'SAFE AREA', 'SAFE AREA', 'SAFE AREA', 'ZONE 1, IIA, T3', 'ZONE 1, IIA, T3'],
'PKG_DESCRIPTION': ['SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP', 'SEA WATER LIFT PUMP']
}

df_lift_pump = pd.DataFrame(data_lift_pump)
print('\n Scope of Supply for WATER LIFT PUMPS (B-5111001A/D AND B-5111002)')
print(df_lift_pump)

# Definição do dataframe do pacote para WATER ELECTROCHLORINATION UNIT

data_electrochlorination = {
'TAG': ['UE-5121501', 'PN-UE-5121501'],
'QTY': ['1 x 100%', '1 x 100%'],
'DESCRIPTION': ['Sea Water Electrochlorination Unit', 'Sea Water Electrochlorination Unit Control Panel'],
'LOCATION': ['M-15', 'M-15'],
'AREA_CLASSIFICATION': ['Outdoor, Safe Area', 'Outdoor, Safe Area'],
'PKG_DESCRIPTION': ['SEA WATER ELECTROCHLORINATION UNIT', 'SEA WATER ELECTROCHLORINATION UNIT']
}

df_electrochlorination = pd.DataFrame(data_electrochlorination)
print('\n Scope of Supply for SEA WATER ELECTROCHLORINATION UNIT')
print(df_electrochlorination)

# Definição do dataframe do pacote para NITROGEN GENERATOR UNITS

data_N2_generation = {
'TAG': ['Z-5241001A/B', 'PN-Z-5241001A/B', 'TBA', 'TBA', 'V-Z-5241001', 'TBA', 'TBA', 'Z-5241002A/B', 'PN-Z-5241002A/B', 'TBA', 'TBA', 'TBA', 'TBA'],
'QTY': ['2 x 100%', '2 x 100%', '2 x 100%', '2 x 100%', '1 x 100%', '1 x 100%', '130 bottles', '2 x 100%', '2 x 100%', '2 x 100%', '2 x 100%', '2 x 100%', '2 x 100%'],
'DESCRIPTION': ['Nitrogen Generator Unit (Filters and membrane modules)', 'Nitrogen Generator Unit Control Panel', 'Nitrogen Compressor', 'Vent Including Silencer', 'Nitrogen Buffer Vessel', 'Nitrogen Intensifier Unit', 'Emergency Bottles Battery (50L) in skidded units', 'Nitrogen Generator Unit for Flare (Filters and membrane modules)', 'Nitrogen Generator Unit for Flare Control Panel', 'Air Compressors', 'Air Dryers with filters', 'Vent Including Silencer', 'Oxygen Content Analyzer'],
'LOCATION': ['M-01', 'M-01', 'M-01', 'M-01', 'M-01', 'M-01', 'M-01', 'Forecastle (N2 Gen. Room)', 'Forecastle (N2 Gen. Room)', 'Forecastle (N2 Gen. Room)', 'Forecastle (N2 Gen. Room)', 'Forecastle (N2 Gen. Room)', 'Control Room'],
'AREA_CLASSIFICATION': ['Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Safe', 'Safe', 'Safe', 'Safe', 'Safe', 'Safe'],
'PKG_DESCRIPTION': ['NITROGEN GENERATOR UNITS', 'NITROGEN GENERATOR UNITS', 'NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS','NITROGEN GENERATOR UNITS']
}

df_N2_generation = pd.DataFrame(data_N2_generation)
print('\n Scope of Supply for NITROGEN GENERATOR UNITS')
print(df_N2_generation)

# Definição do dataframe do pacote para SHELL AND TUBE HEAT EXCHANGERS

data_heat_exchangers = {
'TAG': ['P-1223001A/F', 'P-1223002A/F', 'P-1223003A/B', 'P-1223004', 'P-1251001A/B', 'P-1350001A/B', 'P-5125001', 'P-5135001', 'P-5135002', 'P-5147001', 'P-UC-1225001A/B-01', 'P-UC-1225001A/B-02', 'P-UC-1231001A/C-01', 'P-UC-1231001A/C-02'],
'QTY': ['6 x 25%', '6 x 25%', '2 x 50%', '1 x 100%', '2 x 50%', '2 x 100%', '1 x 100%', '1 x 100%', '1 x 100%', '1 x 100%', '2 x 100%', '2 x 100%', '3 x 50%', '3 x 50%'],
'DESCRIPTION': ['OIL/OIL PRE HEATER', 'PRODUCTION HEATER', 'DILUTION WATER HEATER', 'TEST HEATER', 'INJECTION WATER RECYCLE COOLER', 'STRUCTURAL TANKS RECOVERY GAS COOLER', 'UTILITY HOT WATER HEATER', 'FUEL GAS HEATER', 'FUEL GAS SUPER HEATER', 'HOT WATER DUMP COOLER', 'VRU 1ST STAGE SUCTION COOLER', 'VRU 1ST STAGE DISCHARGE COOLER', 'MAIN GAS COMPRESSOR SUCTION COOLER', 'MAIN GAS COMPRESSOR DISCHARGE COOLER'],
'LOCATION': ['M-10A', 'M-10B', 'M-10C', 'M-10B', 'M-11', 'M-01', 'M-15', 'M-06', 'M-06', 'M-15', 'M-05B', 'M-05B', 'M-05', 'M-05'],
'AREA_CLASSIFICATION': ['M-10C', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'UNCLASSIFIED', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'UNCLASSIFIED', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'UNCLASSIFIED', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'UNCLASSIFIED', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3', 'ZONE 2; GAS GROUP IIA, TEMP CLASS T3'],
'PKG_DESCRIPTION': ['SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS', 'SHELL AND TUBE HEAT EXCHANGERS']
}

df_heat_exchangers = pd.DataFrame(data_heat_exchangers)
print('\n Scope of Supply for SHELL AND TUBE HEAT EXCHANGERS')
print(df_heat_exchangers)

# Definição do dataframe do pacote para PROGRESSIVE CAVITY PUMPS

data_progressive_pump = {
'TAG': ['B-5412001A/B', 'M-B-5412001A/B', 'B-5412002A/B', 'M-B-5412002A/B'],
'QTY': ['2 X 100%', '2 X 100%', '2 X 100%', '2 X 100%'],
'DESCRIPTION': ['HIGH PRESSURE FLARE K. O. DRUM PUMP', 'ELECTRIC MOTOR FOR HIGH PRESSURE FLARE K. O. DRUM PUMP', 'LOW PRESSURE FLARE K. O. DRUM PUMP', 'ELECTRIC MOTOR FOR LOW PRESSURE FLARE K. O. DRUM PUMP'],
'LOCATION': ['M-01 PRODUCTION DECK -LEVEL 1', 'M-01 PRODUCTION DECK -LEVEL 1', 'M-01 PRODUCTION DECK -LEVEL 1', 'M-01 PRODUCTION DECK -LEVEL 1'],
'AREA_CLASSIFICATION': ['ZONE 2, GROUP llA, T3', 'ZONE 2, GROUP llA, T3', 'ZONE 2, GROUP llA, T3', 'ZONE 2, GROUP llA, T3'],
'PKG_DESCRIPTION': ['PROGRESSIVE CAVITY PUMPS','PROGRESSIVE CAVITY PUMPS','PROGRESSIVE CAVITY PUMPS','PROGRESSIVE CAVITY PUMPS']
}

df_progressive_pump = pd.DataFrame(data_progressive_pump)
print('\n Scope of Supply for PROGRESSIVE CAVITY PUMPS')
print(df_progressive_pump)

# Definição do dataframe do pacote para PIG LAUNCHERS AND RECEIVERS

data_PIG = {
'TAG': ['LP-1244001A/J', 'LP-1244002A/E', 'LP-1244003A/G', 'LR-1231001', 'RP-1210001A/B', 'RP-1210002', 'RP-1210003A/B'],
'QTY': ['9', '5', '7', '1', '2', '1', '2'],
'DESCRIPTION': ['Satellite Production Well Pig Launcher', 'PWAGFLEX Well Pig Launcher', 'WAG-LOOP Well Pig Launcher', 'Gas Pipeline Pig Launcher/Receiver', 'Satellite Production Well Pig Receiver', 'PWAGFLEX Well Pig Receiver', 'WAG-LOOP Well Pig Receiver'],
'LOCATION': ['M-09', 'M-09', 'M-09', 'M-09', 'M-09', 'M-09', 'M-09'],
'AREA_CLASSIFICATION': ['Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3', 'Zone 2, Gas Group IIA, T3'],
'PKG_DESCRIPTION': ['PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS','PIG LAUNCHERS AND RECEIVERS']
}

df_PIG = pd.DataFrame(data_PIG)
print('\n Scope of Supply for PIG LAUNCHERS AND RECEIVERS')
print(df_PIG)

# Definição do dataframe do pacote para NON-API 610 CENTRIFUGAL PUMPS

data_non_api_pump = {
'TAG': ['B-5115004A/B', 'M-B-5115004A/B', 'B-5124002A/C', 'M-B-5124002A/C', 'B-5124003', 'M-B-5124003', 'PN-B-5124003', 'B-5125002 A/B', 'M-B-5125002 A/B'],
'QTY': ['2', '2', '3', '3', '1', '1', '1', '2', '2'],
'DESCRIPTION': ['FLARE AND SLOP VESSEL FRESH WATER MAKE-UP PUMP', 'ELECTRIC MOTOR FOR FLARE AND SLOP VESSEL FRESH WATER MAKE-UP PUMP', 'COOLING WATER CIRCULATION PUMP - NON-CLASSIFIED AREA', 'ELECTRIC MOTOR FOR COOLING WATER CIRCULATION PUMP - NON-CLASSIFIED AREA', 'START-UP COOLING WATER PUMP - NON-CLASSIFIED AREA', 'ELECTRIC MOTOR FOR START-UP COOLING WATER PUMP - NON-CLASSIFIED AREA', 'START-UP COOLING WATER PUMP - NON-CLASSIFIED AREA POWER PANEL', 'UTILITY HOT WATER CIRCULATION PUMP', 'ELECTRICAL MOTOR FOR UTILITY HOT WATER CIRCULATION PUMP'],
'LOCATION': ['M-15', 'M-15', 'M-14', 'M-14', 'M-14', 'M-14', 'ENGINE ROOM', 'M-15', 'M-15'],
'AREA_CLASSIFICATION': ['UNCLASSIFIED'] * 2 + ['ZONE 2, GROUP IIA, T3 (for running during ESD-3P)'] * 2 + ['UNCLASSIFIED'] * 5,
'PKG_DESCRIPTION': ['NON-API 610 CENTRIFUGAL PUMPS'] * 9
}

df_non_api_pump = pd.DataFrame(data_non_api_pump)
print('\n Scope of Supply for NON-API 610 CENTRIFUGAL PUMPS')
print(df_non_api_pump)

# Definição do dataframe do pacote para API 610 CENTRIFUGAL PUMPS

data_api_pump = {
'TAG': ['B-1223001 A/B', 'B-1223001 C/D', 'M-B-1223001 A/B', 'M-B-1223001 C/D', 'CF-B-1223001 A/B/C/D', 'B-1223002A/B', 'B-1223002C/D', 'M-B-1223002A/B', 'M-B-1223002C/D', 'B-1223003A/B', 'B-1223003C/D', 'M-B-1223003A/B', 'M-B-1223003C/D', 'CF-B-1223003A/B/C/D', 'B-1223004A/B', 'M-B-1223004A/B', 'B-5124001A/B/C/D', 'M-B-5124001A/B/C/D', 'B-5125001A/B/C', 'M-B-5125001A/B/C', 'B-5125003A/B', 'M-B-5125003A/B', 'B-5331001A/B/C', 'M-B-5331001A/B/C', 'B-5115003A/C (HOLD  - 1)', 'M-B-5115003A/C (HOLD  - 1)'],
'QTY': ['4 X 50%', '4 X 50%', '4', '4', '4', '4 X 50%', '4 X 50%', '4', '4', '4 x 50%', '4 x 50%', '4', '4', '4', '2 X 100%', '2', '4 X 33%', '4', '3 X 50%', '3', '2 X 100%', '2', '3 X 50%', '3', '3 x 50%', '3'],
'DESCRIPTION': ['PRE-OIL DEHYDRATOR  RECIRCULATION WATER  PUMP (OH2 TYPE)', 'PRE-OIL DEHYDRATOR  RECIRCULATION WATER  PUMP (OH2 TYPE)', 'ELECTRIC MOTOR FOR  PRE-OIL DEHYDRATOR  RECIRCULATION WATER  PUMP', 'ELECTRIC MOTOR FOR  PRE-OIL DEHYDRATOR  RECIRCULATION WATER  PUMP', 'FREQUENCY CONVERTER  FOR PRE -OIL  DEHYDRATOR  RECIRCULATION WATER  PUMP', 'OIL DEHYDRATOR  RECIRCULATION WATER  PUMP  (OH2 TYPE)', 'OIL DEHYDRATOR  RECIRCULATION WATER  PUMP  (OH2 TYPE)', 'ELECTRIC MOTOR FOR  OIL DEHYDRATOR  RECIRCULATION WATER  PUMP', 'ELECTRIC MOTOR FOR  OIL DEHYDRATOR  RECIRCULATION WATER  PUMP', 'OIL TRANSFER PUMP  (OH2 TYPE)', 'OIL TRANSFER PUMP  (OH2 TYPE)', 'ELECTRIC MOTOR FOR  OIL TRANSFER PUMP', 'ELECTRIC MOTOR FOR  OIL TRANSFER PUMP', 'FREQUENCY CONVERTER  FOR  OIL TRANSFER   PUMP', 'TEST SEPARATOR PUMP  (OH2 TYPE)', 'ELECTRIC MOTOR FOR  TEST SEPARATOR PUMP', 'COOLING WATER  CIRCULATION PUMP (BB2  TYPE) - CLASSIFIED AREA', 'ELECTRIC MOTOR FOR  COOLING WATER  CIRCULATION PUMP -  CLASSIFIED AREA', 'HOT WATER  CIRCULATION PUMP (OH2  TYPE)', 'ELECTRIC MOTOR FOR  HOT WATER  CIRCULATION PUMP', 'HOT WATER DUMP  COOLER PUMP (OH2  TYPE)', 'ELECTRIC MOTOR FOR  HOT WATER DUMP  COOLER PUMP (OH2  TYPE)', 'PRODUCED WATER  BOOSTER PUMPS (OH2  TYPE)', 'ELECTRIC MOTOR FOR  PRODUCED WATER  BOOSTER PUMPS', 'DILUTION WATER PUMP', 'ELECTRIC MOTOR FOR  DILUTION WATER PUMPS'],
'LOCATION': ['M-10A', 'M-10C', 'M-10A', 'M-10C', 'M-17', 'M-10A', 'M-10C', 'M-10A', 'M-10C', 'M-10A', 'M-10C', 'M-10A', 'M-10C', 'M-17', 'M-10B', 'M-10B', 'M-14', 'M-14', 'M-15', 'M-15', 'M-15', 'M-15', 'M-10B', 'M-10B', 'M-15', 'M-15'],
'AREA_CLASSIFICATION': ['UNCLASSIFIED SAFE AREA'] * 9 + ['ZONE 2 , GROUP  IIA , T3'] * 4 + ['UNCLASSIFIED SAFE AREA'] * 1 + ['ZONE 2 , GROUP  IIA , T3'] * 2 + ['UNCLASSIFIED SAFE AREA'] * 4 + ['ZONE 2 , GROUP IIA , T3 (for running during ESD-3P)'] * 2 + ['ZONE 2 , GROUP  IIA , T3'] * 2 + ['UNCLASSIFIED SAFE AREA'] * 2,
'PKG_DESCRIPTION': ['API 610 CENTRIFUGAL PUMPS'] * 26
}

df_api_pump = pd.DataFrame(data_api_pump)
print('\n Scope of Supply for NON-API 610 CENTRIFUGAL PUMPS')
print(df_api_pump)

list_df_equip = ['df_wip', 'df_lift_pump', 'df_electrochlorination', 'df_N2_generation', 'df_heat_exchangers', 'df_progressive_pump', 'df_PIG', 'df_non_api_pump', 'df_api_pump']


 Scope of Supply for WATER INJECTION PUMPS AND SEA WATER BOOSTER PUMPS
               TAG       QTY                              DESCRIPTION  \
0     B-1251001A/C   3 x 50%             INJECTION WATER BOOSTER PUMP   
1   M-B-1251001A/C     3 set     INJECTION WATER BOOSTER PUMP - MOTOR   
2     B-1251002A/C   3 x 50%                INJECTION WATER MAIN PUMP   
3   M-B-1251002A/C     3 set        INJECTION WATER MAIN PUMP - MOTOR   
4     B-1251003A/B  2 x 100%                    SEAWATER BOOSTER PUMP   
5   M-B-1251003A/B     2 set            SEAWATER BOOSTER PUMP - MOTOR   
6  VV-B-1251002A/C     3 set       HVSD FOR MAIN INJECTION WATER PUMP   
7  PN-B-1251002A/C     3 set  MAIN INJECTION WATER PUMP CONTROL PANEL   

  LOCATION             AREA_CLASSIFICATION        PKG_DESCRIPTION  
0     M-11              Outdoor, Safe Area  WATER INJECTION PUMPS  
1     M-11              Outdoor, Safe Area  WATER INJECTION PUMPS  
2     M-11              Outdoor, Safe Area  WATER INJECTION PUMPS 

#### Sechedules WIP

In [26]:
data_plan_booster = {
'ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28],
'Nome da tarefa': ['BBS 8X12X24J - TAG: B-1251001 A/C - WATER INJECTION BOOSTER PUMPS', 'Contract nº CON-P84-SNT-PROC-0003-24 & CON-P84-SSG-PROC-0004-24', 'Sulzer PO Analysis and Internal project meetings', 'Engineering', 'Main Documents Submittal', 'Documents return from Customer', 'Certified Documents Submittal', 'Final approval from Customer (Major Docs)', 'Pump Main Components & Buy Outs Release', 'Acessories Part List', 'Purchase Orders', 'Bareshaft Pump Acquisition (Sulzer India)', 'Motor / Coupling', 'Baseplate Acquisition', 'Receipt Orders', 'Bareshaft Pump Manufacturing & Performance testing (including MRT)', 'Bareshaft Pump Delivery (India to Brazil - Transit Time - Sea Freight)', 'Electric Motor Delivery', 'Coupling', 'Baseplate Delivery', 'Production & String Test', 'String Test Activities - 01 unit - (Preparation & Test)', 'Package Assembly - Units B, C and D', 'Skids Preservation & Final Inspection Preparation', 'Final Inspection All Units', 'Delivery', 'Equipment packing / invoicing / ExW', 'DAP (Sea Freight) to Seatrium NANTONG SHIPYARD CO LTD (China)'],
'Duration': [395, 1, 4, 245, 35, 30, 120, 60, 10, 50, 5, 5, 5, 5, 260, 210, 50, 250, 200, 230, 50, 20, 40, 7, 3, 65, 5, 60],
'% Complete': [17, 100, 100, 31, 100, 40, 0, 0, 100, 74, 100, 100, 100, 100, 0, 0, 0, 13, 16, 14, 0, 0, 0, 0, 0, 0, 0, 0],
'Start': ['30/09/2024', '30/09/2024', '01/10/2024', '07/10/2024', '07/10/2024', '25/11/2024', '06/01/2025', '23/06/2025', '07/10/2024', '21/10/2024', '21/10/2024', '21/10/2024', '21/10/2024', '21/10/2024', '28/10/2024', '28/10/2024', '18/08/2025', '28/10/2024', '28/10/2024', '28/10/2024', '27/10/2025', '27/10/2025', '27/10/2025', '22/12/2025', '31/12/2025', '05/01/2026', '05/01/2026', '12/01/2026'],
'Finish': ['03/04/2026', '30/09/2024', '04/10/2024', '12/09/2025', '22/11/2024', '03/01/2025', '20/06/2025', '12/09/2025', '18/10/2024', '27/12/2024', '25/10/2024', '25/10/2024', '25/10/2024', '25/10/2024', '24/10/2025', '15/08/2025', '24/10/2025', '10/10/2025', '01/08/2025', '12/09/2025', '02/01/2026', '21/11/2025', '19/12/2025', '30/12/2025', '02/01/2026', '03/04/2026', '09/01/2026', '03/04/2026']
}

df_plan_booster = pd.DataFrame(data_plan_booster)
df_plan_booster['Start'] = pd.to_datetime(df_plan_booster['Start'], format='%d/%m/%Y')
df_plan_booster['Finish'] = pd.to_datetime(df_plan_booster['Finish'], format='%d/%m/%Y')

display(df_plan_booster)

data_plan_sea_booster = {
'ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28],
'Nome da tarefa': ['CD 20X20x25XZ - TAG: B-1251003A/B - SEA WATER BOOSTER PUMPS', 'Contract nº CON-P84-SNT-PROC-0003-24 & CON-P84-SSG-PROC-0004-24', 'Sulzer PO Analysis and Internal project meetings', 'Engineering', 'Main Documents Submittal', 'Documents return from Customer', 'Certified Documents Submittal', 'Final approval from Customer (Major Docs)', 'Pump Main Components & Buy Outs Release', 'Acessories Part List', 'Purchase Orders', 'Bareshaft Pump Acquisition (Sulzer China)', 'Motor / Coupling', 'Baseplate Acquisition', 'Receipt Orders', 'Bareshaft Pump Manufacturing & Performance testing (including MRT)', 'Bareshaft Pump Delivery (India to Brazil - Transit Time - Sea Freight)', 'Electric Motor Delivery', 'Coupling', 'Baseplate Delivery', 'Production & String Test', 'String Test Activities - 01 unit - (Preparation & Test)', 'Package Assembly', 'Skids Preservation & Final Inspection Preparation', 'Final Inspection All Units', 'Delivery', 'Equipment packing / invoicing / ExW', 'DAP (Sea Freight) to Seatrium NANTONG SHIPYARD CO LTD (China)'],
'Duration': [395, 1, 4, 245, 35, 30, 120, 60, 10, 50, 5, 5, 5, 5, 265, 215, 50, 250, 200, 240, 50, 20, 40, 7, 3, 65, 5, 60],
'% Complete': [17, 100, 100, 31, 100, 40, 0, 0, 100, 74, 100, 100, 100, 100, 13, 15, 0, 13, 16, 13, 0, 0, 0, 0, 0, 0, 0, 0],
'Start': ['30/09/24', '30/09/24', '01/10/24', '07/10/24', '07/10/24', '25/11/24', '06/01/25', '23/06/25', '07/10/24', '21/10/24', '21/10/24', '21/10/24', '21/10/24', '21/10/24', '28/10/24', '28/10/24', '25/08/25', '28/10/24', '28/10/24', '28/10/24', '27/10/25', '03/11/25', '27/10/25', '22/12/25', '31/12/25', '05/01/26', '05/01/26', '12/01/26'],
'Finish': ['03/04/26', '30/09/24', '04/10/24', '12/09/25', '22/11/24', '03/01/25', '20/06/25', '12/09/25', '18/10/24', '27/12/24', '25/10/24', '25/10/24', '25/10/24', '25/10/24', '31/10/25', '22/08/25', '31/10/25', '10/10/25', '01/08/25', '26/09/25', '02/01/26', '28/11/25', '19/12/25', '30/12/25', '02/01/26', '03/04/26', '09/01/26', '03/04/26']
}

df_plan_sea_booster = pd.DataFrame(data_plan_sea_booster)
df_plan_sea_booster['Start'] = pd.to_datetime(df_plan_sea_booster['Start'], format='%d/%m/%y')
df_plan_sea_booster['Finish'] = pd.to_datetime(df_plan_sea_booster['Finish'], format='%d/%m/%y')

display(df_plan_sea_booster)


data_plan_wip = {
'ID': [1, 2, 3, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 64, 65],
'Nome da tarefa': ['GSG 200-400 TAG: B-1251002 A/C - WATER INJECTION MAIN PUMPS', 'Contract nº CON-P84-SNT-PROC-0003-24 & CON-P84-SSG-PROC-0004-24', 'Sulzer PO Analysis and Internal project meetings', 'Engineering', 'Buy Outs Release', 'Forgings & Castings Release', 'Pump Project (Pump BOM)', 'Package Project (Package BOM)', 'Purchasing (Main Items)', 'Mechanical Seal', 'Mechanical Seal + API 11/62 Acquisition', 'Manufacturing + Testing', 'Delivery at SBR', 'Electric Motor', 'Electric Motor Acquisition', 'Manufacturing', 'Supplier Final Tests and Inspection', 'Delivery at SBR', 'Coupling', 'Coupling acquisition', 'Manufacturing', 'Delivery at SBR', 'Variable Hydraulic Speed Coupling (HSVC)', 'HSVC Acquisition', 'Manufacturing', 'Supplier Final Tests and Inspection', 'Delivery  EXW Germany', 'Delivery at SBR (Sea Freight)', 'Electric Panel', 'Panel Acquisition', 'Manufacturing', 'Supplier Final Tests and Inspection', 'Delivery at SBR', 'Baseplate', 'Baseplate Acquisition', 'Manufacturing', 'Delivery at SBR', 'Pumps Production and Test', 'Pump Forgings Acquisition', 'Pump Barrel Production + Delivery', 'Barrel Machining', 'Pump Assembly to Hydro Test', 'Internal Hydro Test', 'Witness Hydro Test', 'Pump Assembly to Performance Test', 'Skid Assembly Activities Piping Activities', 'Internal Performance (03X) // String Test (01X)', 'Witness Performance (03X) // String Test (01X)', 'Skids Preservation & Final Inspection Preparation', 'Final Inspection', 'Expedition', 'Equipment packing / invoicing / EwW Delivery', 'DAP (Sea Freight) to Seatrium NANTONG SHIPYARD CO LTD (China)', 'Control Panel packing / invoicing / EwW Delivery', 'DAP (Sea Freight) to CIMC Yard (China)'],
'Duration': [395, 1, 4, 130, 13, 13, 80, 50, 377, 165, 5, 140, 20, 221, 5, 190, 20, 5, 245, 5, 220, 20, 275, 5, 190, 30, 5, 45, 264, 20, 220, 20, 4, 187, 20, 160, 7, 313, 5, 185, 30, 5, 5, 5, 12, 40, 7, 6, 5, 5, 64, 4, 60, 4, 60],
'% Complete': [19, 100, 100, 54, 100, 100, 73, 0, 16, 25, 100, 26, 0, 19, 100, 19, 0, 0, 17, 100, 16, 0, 15, 100, 19, 0, 0, 0, 16, 100, 10, 0, 0, 22, 100, 13, 0, 13, 100, 19, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
'Start': ['30/09/24', '30/09/24', '01/10/24', '01/10/24', '07/10/24', '07/10/24', '01/10/24', '21/01/25', '24/10/24', '24/10/24', '24/10/24', '31/10/24', '15/05/25', '24/10/24', '24/10/24', '31/10/24', '24/07/25', '22/08/25', '24/10/24', '24/10/24', '31/10/24', '04/09/25', '24/10/24', '24/10/24', '31/10/24', '24/07/25', '04/09/25', '11/09/25', '24/10/24', '24/10/24', '21/11/24', '25/09/25', '23/10/25', '24/10/24', '24/10/24', '21/11/24', '03/07/25', '24/10/24', '24/10/24', '31/10/24', '17/07/25', '28/08/25', '04/09/25', '11/09/25', '18/09/25', '09/10/25', '04/12/25', '15/12/25', '23/12/25', '30/12/25', '06/01/26', '06/01/26', '12/01/26', '06/01/26', '12/01/26'],
'Finish': ['03/04/26', '30/09/24', '04/10/24', '31/03/25', '23/10/24', '23/10/24', '20/01/25', '31/03/25', '03/04/26', '11/06/25', '30/10/24', '14/05/25', '11/06/25', '28/08/25', '30/10/24', '23/07/25', '21/08/25', '28/08/25', '01/10/25', '30/10/24', '03/09/25', '01/10/25', '12/11/25', '30/10/24', '23/07/25', '03/09/25', '10/09/25', '12/11/25', '28/10/25', '20/11/24', '24/09/25', '22/10/25', '28/10/25', '11/07/25', '20/11/24', '02/07/25', '11/07/25', '05/01/26', '30/10/24', '16/07/25', '27/08/25', '03/09/25', '10/09/25', '17/09/25', '03/10/25', '03/12/25', '12/12/25', '22/12/25', '29/12/25', '05/01/26', '03/04/26', '09/01/26', '03/04/26', '09/01/26', '03/04/26']
}

df_plan_wip = pd.DataFrame(data_plan_wip)
df_plan_wip['Start'] = pd.to_datetime(df_plan_wip['Start'], format='%d/%m/%y')
df_plan_wip['Finish'] = pd.to_datetime(df_plan_wip['Finish'], format='%d/%m/%y')

display(df_plan_wip)


,ID,Nome da tarefa,Duration,% Complete,Start,Finish
0,1,BBS 8X12X24J - TAG: B-1251001 A/C - WATER INJE...,395,17,2024-09-30,2026-04-03
1,2,Contract nº CON-P84-SNT-PROC-0003-24 & CON-P84...,1,100,2024-09-30,2024-09-30
2,3,Sulzer PO Analysis and Internal project meetings,4,100,2024-10-01,2024-10-04
3,4,Engineering,245,31,2024-10-07,2025-09-12
4,5,Main Documents Submittal,35,100,2024-10-07,2024-11-22
5,6,Documents return from Customer,30,40,2024-11-25,2025-01-03
6,7,Certified Documents Submittal,120,0,2025-01-06,2025-06-20
7,8,Final approval from Customer (Major Docs),60,0,2025-06-23,2025-09-12
8,9,Pump Main Components & Buy Outs Release,10,100,2024-10-07,2024-10-18
9,10,Acessories Part List,50,74,2024-10-21,2024-12-27


,ID,Nome da tarefa,Duration,% Complete,Start,Finish
0,1,CD 20X20x25XZ - TAG: B-1251003A/B - SEA WATER ...,395,17,2024-09-30,2026-04-03
1,2,Contract nº CON-P84-SNT-PROC-0003-24 & CON-P84...,1,100,2024-09-30,2024-09-30
2,3,Sulzer PO Analysis and Internal project meetings,4,100,2024-10-01,2024-10-04
3,4,Engineering,245,31,2024-10-07,2025-09-12
4,5,Main Documents Submittal,35,100,2024-10-07,2024-11-22
5,6,Documents return from Customer,30,40,2024-11-25,2025-01-03
6,7,Certified Documents Submittal,120,0,2025-01-06,2025-06-20
7,8,Final approval from Customer (Major Docs),60,0,2025-06-23,2025-09-12
8,9,Pump Main Components & Buy Outs Release,10,100,2024-10-07,2024-10-18
9,10,Acessories Part List,50,74,2024-10-21,2024-12-27


,ID,Nome da tarefa,Duration,% Complete,Start,Finish
0,1,GSG 200-400 TAG: B-1251002 A/C - WATER INJECTI...,395,19,2024-09-30,2026-04-03
1,2,Contract nº CON-P84-SNT-PROC-0003-24 & CON-P84...,1,100,2024-09-30,2024-09-30
2,3,Sulzer PO Analysis and Internal project meetings,4,100,2024-10-01,2024-10-04
3,5,Engineering,130,54,2024-10-01,2025-03-31
4,6,Buy Outs Release,13,100,2024-10-07,2024-10-23
5,7,Forgings & Castings Release,13,100,2024-10-07,2024-10-23
6,8,Pump Project (Pump BOM),80,73,2024-10-01,2025-01-20
7,9,Package Project (Package BOM),50,0,2025-01-21,2025-03-31
8,11,Purchasing (Main Items),377,16,2024-10-24,2026-04-03
9,12,Mechanical Seal,165,25,2024-10-24,2025-06-11


In [27]:
def count_qty(df):
    count = 0
    for qty_str in df['QTY']:
        parts = qty_str.split()
        # Verificar se a primeira parte é um número válido
        if parts[0].isdigit():
            count += int(parts[0])

    return count

total = 0
data_for_df_equip = []  # Lista para armazenar os dados do novo DataFrame

for df_name in list_df_equip:
   
    df = globals()[df_name]
    qty_count = count_qty(df)
    # Obter o nome do pacote da coluna PKG_DESCRIPTION
    # Assumindo que todos os valores nesta coluna são iguais para um dado DataFrame
    pkg_name = df['PKG_DESCRIPTION'].iloc[0] if not df['PKG_DESCRIPTION'].empty else "Nome do pacote não disponível"

    print(f"Total de equipamentos do pacote {pkg_name}: {qty_count}")
    total += qty_count
    # Adicionar dados para o novo DataFrame 
    data_for_df_equip.append({'PKG_DESCRIPTION': pkg_name, 'QTY_EQUIPMENT': qty_count}) 

print(f"\nTotal de equipamentos: {total}")

# Criar o novo DataFrame df_equip 
df_equip = pd.DataFrame(data_for_df_equip)

Total de equipamentos do pacote WATER INJECTION PUMPS: 22
Total de equipamentos do pacote SEA WATER LIFT PUMP: 21
Total de equipamentos do pacote SEA WATER ELECTROCHLORINATION UNIT: 2
Total de equipamentos do pacote NITROGEN GENERATOR UNITS: 152
Total de equipamentos do pacote SHELL AND TUBE HEAT EXCHANGERS: 33
Total de equipamentos do pacote PROGRESSIVE CAVITY PUMPS: 8
Total de equipamentos do pacote PIG LAUNCHERS AND RECEIVERS: 27
Total de equipamentos do pacote NON-API 610 CENTRIFUGAL PUMPS: 17
Total de equipamentos do pacote API 610 CENTRIFUGAL PUMPS: 90

Total de equipamentos: 372


### Importando os dados via Aida

#### Autenticação no AIDA

In [28]:

authority = 'https://login.microsoftonline.com/5b6f6241-9a57-4be4-8e50-1dfa72e79a57'
client_id = '404e72ec-2b28-459f-b167-40730d7610ae'
resource_id = client_id

context = adal.AuthenticationContext(authority)
code = context.acquire_user_code(resource_id, client_id)

print(code['message'])

token = context.acquire_token_with_device_code(resource_id, code, client_id)
print(token['accessToken'])

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code HUJGDDFWG to authenticate.
eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImltaTBZMnowZFlLeEJ0dEFxS19UdDVoWUJUayIsImtpZCI6ImltaTBZMnowZFlLeEJ0dEFxS19UdDVoWUJUayJ9.eyJhdWQiOiI0MDRlNzJlYy0yYjI4LTQ1OWYtYjE2Ny00MDczMGQ3NjEwYWUiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC81YjZmNjI0MS05YTU3LTRiZTQtOGU1MC0xZGZhNzJlNzlhNTcvIiwiaWF0IjoxNzQwNTU3MDY5LCJuYmYiOjE3NDA1NTcwNjksImV4cCI6MTc0MDU2MTYxMywiYWNyIjoiMSIsImFpbyI6IkFZUUFlLzhaQUFBQVlnY0dZcmx3VEFKdGRNdy83REJ3SmdiTCs5aDBia3hTbGp2YTdydkcvVzE5NUtobCtxY3JDOTQ0TkFtVU05czgyL0xoTEVyZkhjRkFIZ1cra2RYSEM2MEg4LzMvOEdMUGR6LytPZ1JlNGtZLzBsU1lEb21zT2hKKzJjUDZCTFRoeW1Pb2xocHpNZmlFSUxaRmhLdnFvM3BnVlpEbjdaUVdtN1FMN1k5LzZrVT0iLCJhbXIiOlsiZmlkbyIsIm1mYSJdLCJhcHBpZCI6IjQwNGU3MmVjLTJiMjgtNDU5Zi1iMTY3LTQwNzMwZDc2MTBhZSIsImFwcGlkYWNyIjoiMCIsImZhbWlseV9uYW1lIjoiU2lsdmVzdHJlIiwiZ2l2ZW5fbmFtZSI6IkNlc2FyIiwiaXBhZGRyIjoiMTI5LjEyNi42Ny4xMzQiLCJuYW1lIjoiQ2VzYXIgQXVndXN0byBQZXJlaXJh

#### Requisição de todos os documentos do Projeto da P84

In [29]:
format_request = 'application/json'

api_url = 'https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_view_integra_doc'

size_page = 100
from_page = 1

count = 0

df = pd.DataFrame()

# Adicionando o filtro para o projeto P84 
filter_project = "project eq 'PRJ-0000000017'" 

while ((count == 0 ) or count > (size_page*(from_page - 1))):

    head = {'Authorization': 'Bearer '+ token['accessToken']}

    # Modificando a URL para incluir o filtro 
    current_url = (f"{api_url}?$format={format_request}&$count=true&$top={size_page}" 
                   f"&$skip={size_page*(from_page-1)}&$filter={filter_project}") 
    #current_url = api_url + '?$format=' + format_request + '&$count=true&$top=' + str(size_page) + '&$skip=' + str(size_page*(from_page-1))

    print(current_url)
    from_page = from_page + 1
    jsonStr = requests.get(current_url, headers = head).json()

    if (count == 0):
        count = jsonStr['@odata.count'];

    df = pd.concat([df, pd.read_json(json.dumps(jsonStr['value']), orient='records')], ignore_index=True)
    df_integra = df.copy()

print(df_integra.info())


https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_view_integra_doc?$format=application/json&$count=true&$top=100&$skip=0&$filter=project eq 'PRJ-0000000017'
https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_view_integra_doc?$format=application/json&$count=true&$top=100&$skip=100&$filter=project eq 'PRJ-0000000017'
https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_view_integra_doc?$format=application/json&$count=true&$top=100&$skip=200&$filter=project eq 'PRJ-0000000017'
https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_view_integra_doc?$format=application/json&$count=true&$top=100&$skip=300&$filter=project eq 'PRJ-0000000017'
https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_view_integra_doc?$format=application/json&$count=true&$top=100&$skip=400&$filter=project eq 'PRJ-0000000017'
https://app-prd-ty49.azurew

#### Verifica se o documento está atrasado

In [30]:
import datetime 
 
# Função para calcular dias úteis 
def dias_uteis(data_inicial, data_final): 
    dias = (data_final - data_inicial).days 
    uteis = sum(1 for dia in (data_inicial + datetime.timedelta(n) for n in range(dias)) if dia.weekday() < 5) 
    return uteis 
 
# Criar a nova coluna para documentos do tipo "TECHNICAL ENGINEERING DOCUMENT" 
df_integra.loc[df_integra['object_type'] == 'TECHNICAL ENGINEERING DOCUMENT', 'atrasado'] = ( 
    (df_integra['creation_date'].apply(lambda x: dias_uteis(datetime.datetime.strptime(x, '%d/%m/%Y %H:%M:%S'), datetime.datetime.now())) > 10) & 
    (~df_integra['reference_document'].str.contains('VDA', na=False)) 
) 
 
# Criar a nova coluna para documentos do tipo "VDA" 
df_integra.loc[df_integra['object_type'] == 'VDA', 'atrasado'] = ( 
    (df_integra['creation_date'].apply(lambda x: dias_uteis(datetime.datetime.strptime(x, '%d/%m/%Y %H:%M:%S'), datetime.datetime.now())) > 10) & 
    (df_integra['workflow_status'] != 'FINALIZADO') 
) 
 
# Preencher os valores NaN com False 
df_integra['atrasado'] = df_integra['atrasado'].fillna(False)

# Criar a nova coluna com a quantidade de dias úteis desde a 'creation_date' 
df_integra['dias_uteis'] = df_integra['creation_date'].apply(lambda x: dias_uteis(datetime.datetime.strptime(x, '%d/%m/%Y %H:%M:%S'), datetime.datetime.now())) 



#### Cria um dataframe somente com a última revisão

In [31]:

# cria um dataframe chamado df_integra_last somente com os documentos na última versão e não revisão:
df_integra_last = df_integra.loc[df_integra.groupby('document')['version'].idxmax()] 
# df_integra_last = df_integra[df_integra['latest_revision'] == 'Sim']

#### Requisção de todos os Workflows do projeto P84 e que contém o trigrama da Sulzer SJA

In [32]:

format_request = 'application/json' 
 
api_url = 'https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_mv_integra_wf' 
 
size_page = 100 
from_page = 1 
 
count = 0 
 
df_wf = pd.DataFrame() 

# Adicionando o filtro para o projeto P84 e para código do documento SJA - Sulzer
filter_project = "project eq 'PRJ-0000000017' and contains(document, 'SJA')"

while ((count == 0) or count > (size_page * (from_page - 1))): 
    head = {'Authorization': 'Bearer ' + token['accessToken']} 
 
    # Modificando a URL para incluir o filtro 
    current_url = (f"{api_url}?$format={format_request}&$count=true&$top={size_page}" 
                   f"&$skip={size_page*(from_page-1)}&$filter={filter_project}")

    print(current_url) 
    from_page = from_page + 1 
    jsonStr = requests.get(current_url, headers=head).json() 
 
    if (count == 0): 
        count = jsonStr['@odata.count'] 
 
    df_wf = pd.concat([df_wf, pd.read_json(json.dumps(jsonStr['value']), orient='records')], ignore_index=True) 
 

https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_mv_integra_wf?$format=application/json&$count=true&$top=100&$skip=0&$filter=project eq 'PRJ-0000000017' and contains(document, 'SJA')
https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_mv_integra_wf?$format=application/json&$count=true&$top=100&$skip=100&$filter=project eq 'PRJ-0000000017' and contains(document, 'SJA')
https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_mv_integra_wf?$format=application/json&$count=true&$top=100&$skip=200&$filter=project eq 'PRJ-0000000017' and contains(document, 'SJA')
https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_mv_integra_wf?$format=application/json&$count=true&$top=100&$skip=300&$filter=project eq 'PRJ-0000000017' and contains(document, 'SJA')
https://app-prd-ty49.azurewebsites.net/aida/v1/service/dt0001_prd_w1li_dbintegra_w1li_mv_integra_wf?$format=applic

### Importando arquivos

In [35]:
ld_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\I-LD-3010.2S-1251-911-SJA-001_A.xlsx'
ld_file_VDA = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LD Sulzer_have_a_comment.xlsx'
escopo_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2024-08-30 P84 Procurement Map - Topside&Hull_PE_Alterado Robson_Ajustes Casco.xlsx'
mapa_topsideP84_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2025-02-24 P84 Procurement Map Topside.xlsx'
mapa_topsideP85_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2025-02-24 P85 Procurement Map Topside.xlsx'
mapa_hullP84_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2025-02-21 P84 Procurement Map Hull.xlsx'
mapa_hullP85_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2025-02-21 P85 Procurement Map Hull.xlsx'
mapa_orbis_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2025-02-26 orbis.xlsx'
comunication_matrix_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2024-03-10 - P84-P85 Package Communication Matrix.xlsx'
plan_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\P84-RW02.xlsx'
trigram = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\P84.P85 Vendor Registration List 25.02.05.xlsx'

df_LD_original = pd.read_excel(ld_file, sheet_name="VDRL")
df_VDA = pd.read_excel(ld_file_VDA, sheet_name="LD")
df_escopo = pd.read_excel(escopo_file, sheet_name="P84 Topside Map")
df_mapa_topsideP84 = pd.read_excel(mapa_topsideP84_file, sheet_name="P84 Topside Map")
df_mapa_topsideP85 = pd.read_excel(mapa_topsideP85_file, sheet_name="P85 Topside Map")
df_mapa_hullP84 = pd.read_excel(mapa_hullP84_file, sheet_name="P84 Hull – Map")
df_mapa_hullP85 = pd.read_excel(mapa_hullP85_file, sheet_name="P85 Hull – Map")
df_mapa_orbis = pd.read_excel(mapa_orbis_file, sheet_name="Export")
df_mc = pd.read_excel(comunication_matrix_file, sheet_name="Package Matrix")
df_plan = pd.read_excel(plan_file, sheet_name="Plan")
df_trigram1 = pd.read_excel(trigram, sheet_name="VRL (Topside)")
df_trigram2 = pd.read_excel(trigram, sheet_name="VRL (Hull)")
# Concatena os dois DataFrames verticalmente 
df_trigram = pd.concat([df_trigram1, df_trigram2], ignore_index=True) 


c:\Users\ELXY\Documents\Codigos\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


#### Pré-Processamento dos dados

In [36]:
# Limpeza inicial removendo cabeçalho do excel 
df_LD_original.columns = df_LD_original.iloc[6] 
df_LD_original = df_LD_original.iloc[7:].reset_index(drop=True) 
 
# Definir as colunas que queremos manter 
colunas_para_manter = ['CLIENT DOCUMENT NUMBER', 'DOCUMENT TITLE', 'PLANNED DATE'] 
 
# Criar um novo DataFrame apenas com as colunas que queremos 
df_LD = df_LD_original[colunas_para_manter].copy()
# Definir os novos nomes das colunas 
novos_nomes = [ 
    'CLIENT_DOCUMENT', 
    'DOCUMENT_TITLE', 
    'PLANNED_DATE_1', 
    'PLANNED_DATE_2', 
    'PLANNED_DATE_3', 
    'PLANNED_DATE_4' 
]

df_LD.columns = novos_nomes

# Exibir as primeiras linhas do DataFrame limpo e renomeado 
display(df_LD.head())
# Exibir informações sobre o DataFrame limpo 
#print(df_LD.info())

df_escopo.rename(columns = {"Package Engineer Petrobras":"Package_Engineer_Petrobras"}, inplace = True)
df_escopo.rename(columns = {"Type_(Super_Critical_/_Critical_/_NA)":"Critical"}, inplace = True)
df_mapa_orbis.rename(columns = {"PKG DESCRIPTION":"PKG_DESCRIPTION", "Pkg ID":"Pkg_ID" }, inplace = True)
df_mc.rename(columns={"Package Engineer_Petrobras":"Package_Engineer_Petrobras"}, inplace = True)
df_mc.rename(columns={"Package Engineer_Engineering":"Package_Engineer_Engineering"}, inplace = True)
df_mc.rename(columns={"EPC_Responsible_(Expeditor)":"Package_Expeditor"}, inplace = True)


# limpeza de espaços em branco para todas as colunas de cada DataFrame da lista abaixo:
# Lista de DataFrames 
dataframes = [df_LD_original, df_LD_original, df_VDA, df_escopo, df_mapa_topsideP84,  
              df_mapa_topsideP85, df_mapa_hullP84, df_mapa_hullP85, df_mapa_orbis,  
              df_mc, df_plan, df_trigram] 
 
# Função para limpar espaços em branco 
def limpar_espacos(df): 
    return df.applymap(lambda x: x.strip() if isinstance(x, str) else x) 
 
# Aplicar a limpeza para cada DataFrame na lista 
for i, df in enumerate(dataframes): 
    dataframes[i] = limpar_espacos(df)
 
# Atribuir os DataFrames limpos de volta às variáveis originais 
df_LD_original, df_LD_original, df_VDA, df_escopo, df_mapa_topsideP84, \
df_mapa_topsideP85, df_mapa_hullP84, df_mapa_hullP85, df_mapa_orbis, \
df_mc, df_plan, df_trigram = dataframes
 
print("Limpeza de espaços em branco concluída para todos os DataFrames.")

,CLIENT_DOCUMENT,DOCUMENT_TITLE,PLANNED_DATE_1,PLANNED_DATE_2,PLANNED_DATE_3,PLANNED_DATE_4
0,I-LD-3010.2S-1251-911-SJA-001,LIST OF VENDOR'S DOCUMENTS AND DRAWINGS FOR AL...,2024-10-25 00:00:00,2024-12-09 00:00:00,NaN,NaN
1,I-CE-3010.2S-1251-700-SJA-001,HAZARDOUS AREA CERTIFICATES FOR ALL ELECTRICAL...,2025-05-29 00:00:00,2025-07-13 00:00:00,NaN,NaN
2,I-CE-3010.2S-1251-700-SJA-002,HAZARDOUS AREA CERTIFICATES FOR ALL INSTRUMENT...,2025-05-29 00:00:00,2025-07-13 00:00:00,NaN,NaN
3,I-CE-3010.2S-1251-700-SJA-101,HAZARDOUS AREA CERTIFICATES FOR ALL ELECTRICAL...,2025-05-29 00:00:00,2025-07-13 00:00:00,NaN,NaN
4,I-CE-3010.2S-1251-700-SJA-102,HAZARDOUS AREA CERTIFICATES FOR ALL INSTRUMENT...,2025-05-29 00:00:00,2025-07-13 00:00:00,NaN,NaN


Limpeza de espaços em branco concluída para todos os DataFrames.


## MAPA DE SUPRIMENTOS:

### Percorre os 4 dataframes e atualiza o df_escopo 

In [37]:
# Verificar e converter tipos de dados das colunas para datetime
date_columns = [
    "PO_Issue_Date_Plan", "PO_Issue_Date_Replan", "PO_Issue_Date_Actual",
    "Fat_Date_Plan", "Fat_Date_Replan", "Fat_Date_Actual",
    "Delivery_On_Site_Date_Plan", "Delivery_On_Site_Date_Replan", "Delivery_On_Site_Date_Actual"
]

for col in date_columns:
    df_escopo[col] = pd.to_datetime(df_escopo[col], errors='coerce')


lista_dfs = ["df_mapa_topsideP84", "df_mapa_topsideP85", "df_mapa_hullP84", "df_mapa_hullP85"]

for item in lista_dfs:
    df = eval(item)  # Converte o nome da variável em um objeto DataFrame
    print(f"Processing DataFrame: {item}")
    
    for row in df.itertuples():
        pkg_description = row.PKG_DESCRIPTION
        item_tag = row.item_tag
        tbe_number = row.TBE_Number
        po_number = row.PO_Number
        po_issue_date_plan = row.PO_Issue_Date_Plan
        po_issue_date_replan = row.PO_Issue_Date_Replan
        po_issue_date_actual = row.PO_Issue_Date_Actual
        fat_date_plan = row.Fat_Date_Plan
        fat_date_replan = row.Fat_Date_Replan
        fat_date_actual = row.Fat_Date_Actual
        delivery_on_site_date_plan = row.Delivery_On_Site_Date_Plan
        delivery_on_site_date_replan = row.Delivery_On_Site_Date_Replan
        delivery_on_site_date_actual = row.Delivery_On_Site_Date_Actual
        
        # usando a função .loc para atualizar os valores
        # df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description) & (df_escopo['item_tag'] == item_tag) , "PO_Number"] = pkg_engineer
        if tbe_number:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "TBE_Number"] = tbe_number
        if po_number:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "PO_Number"] = po_number
        if po_issue_date_plan:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "PO_Issue_Date_Plan"] = po_issue_date_plan
        if po_issue_date_replan:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "PO_Issue_Date_Replan"] = po_issue_date_replan
        if po_issue_date_actual:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "PO_Issue_Date_Actual"] = po_issue_date_actual
        if fat_date_plan:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "Fat_Date_Plan"] = fat_date_plan
        if fat_date_replan:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "Fat_Date_Replan"] = fat_date_replan
        if fat_date_actual:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "Fat_Date_Actual"] = fat_date_actual
        if delivery_on_site_date_plan:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "Delivery_On_Site_Date_Plan"] = delivery_on_site_date_plan
        if delivery_on_site_date_replan:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "Delivery_On_Site_Date_Replan"] = delivery_on_site_date_replan
        if delivery_on_site_date_actual:
            df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "Delivery_On_Site_Date_Actual"] = delivery_on_site_date_actual

print("Atualiza o pkg_id.")
for row in df_mapa_orbis.itertuples():
    pkg_description = row.PKG_DESCRIPTION
    pkg_ID = row.Pkg_ID
    # if pkg_ID:
    df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "Pkg_ID"] = pkg_ID

Processing DataFrame: df_mapa_topsideP84
Processing DataFrame: df_mapa_topsideP85
Processing DataFrame: df_mapa_hullP84
Processing DataFrame: df_mapa_hullP85
Atualiza o pkg_id.


### Verifica se a MR e o TBE foram emitidos

In [38]:
print("Verifica se MR e TBE foram emitidos...")
for row in df_integra.itertuples():
    document_emitido = row.document

    df_escopo.loc[(df_escopo['MR_Number'] == document_emitido), "MR_Emitida"] = "Sim"
    df_escopo.loc[(df_escopo['TBE_Number'] == document_emitido), "TBE_Emitida"] = "Sim"

Verifica se MR e TBE foram emitidos...


#### Verifica se já existe Package Engineer da Seatrium e trigrama

In [39]:
print("Verifica se já existe Package Engineer da Seatrium...") 
for row in df_mc.itertuples():
    pkg_description = row.PKG_DESCRIPTION
    pkg_leader = row.Package_Expeditor
    pkg_engineer = row.Package_Engineer_Engineering
    pkg_petrobras = row.Package_Engineer_Petrobras
    df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "Package_Seatrium"] = pkg_engineer
    df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == pkg_description), "Package_Expeditor"] = pkg_leader
 
df_escopo["Package_Seatrium"] = df_escopo["Package_Seatrium"].fillna("")

print("Verifica se já existe Trigrama...") 
for row in df_trigram.itertuples():
    package = row.PACKAGE
    trigram = row.TRIGRAM
    df_escopo.loc[(df_escopo['PKG_DESCRIPTION'] == package), "Trigram"] = trigram



Verifica se já existe Package Engineer da Seatrium...
Verifica se já existe Trigrama...


## LD da Sulzer

#### Filtra os documentos da Sulzer já emitidos

In [40]:
# Cria uma função para adicionar o padrão regex:
def criar_regex_pattern(*textos_busca): 
    return ''.join(f"(?=.*{texto})" for texto in textos_busca if texto) 
 
texto_busca1 = "1251"
texto_busca2 = "SJA" 
texto_busca3 = ""  # Por exemplo posso deixar o terceiro texto de busca em branco 
texto_busca4 = ""
 
# Criando a expressão regular 
regex_pattern = criar_regex_pattern(texto_busca1, texto_busca2, texto_busca3, texto_busca4) 

# Filtrando o DataFrame 
df_filtrado = df_integra_last[df_integra_last['document'].str.contains(regex_pattern, regex=True, na=False)]

display(df_filtrado)


,document,latest_revision,revision,version,title,release_status_vote,project,mnemonic_project,folder_path,creation_user,...,workflow_status,updated_by,last_update_date,emission_date,document_status,organization,url,object_type,atrasado,dias_uteis
27489,I-CE-3010.2S-1251-910-SJA-001,Sim,B,1,QUALITY MANAGEMENT SYSTEM CERTIFICATE FOR ALL ...,NOT COMMENTED,PRJ-0000000017,P84,ATAPU 2 - P-84\02 - VENDOR DOCUMENTS\TOPSIDE\1...,shaikmeera.mohiddin,...,FINALIZADO,cesar.silvestre,28/12/2024 11:31:03,12/12/2024 04:45:51,CURRENT,KIB,https://integra-ext.petrobras.com.br/INTEGRA/#...,TECHNICAL ENGINEERING DOCUMENT,False,54
27372,I-CR-3010.2S-1251-911-SJA-001,Sim,B,1,FABRICATION PRODUCTION SCHEDULE FOR ALL TAGS,COMMENTS ADDED,PRJ-0000000017,P84,ATAPU 2 - P-84\02 - VENDOR DOCUMENTS\TOPSIDE\1...,shaikmeera.mohiddin,...,FINALIZADO,cesar.silvestre,19/12/2024 19:16:00,06/12/2024 04:22:41,CURRENT,KIB,https://integra-ext.petrobras.com.br/INTEGRA/#...,TECHNICAL ENGINEERING DOCUMENT,False,58
26905,I-DE-3010.2S-1251-311-SJA-001,Sim,A,1,GENERAL ARRANGEMENT DRAWING FOR B-1251001A/B/C,None,PRJ-0000000017,P84,ATAPU 2 - P-84\02 - VENDOR DOCUMENTS\TOPSIDE\1...,shaikmeera.mohiddin,...,None,shaikmeera.mohiddin,23/01/2025 06:53:09,23/01/2025 06:50:18,CURRENT,KIB,https://integra-ext.petrobras.com.br/INTEGRA/#...,TECHNICAL ENGINEERING DOCUMENT,False,24
26904,I-DE-3010.2S-1251-311-SJA-002,Sim,0,1,PUMP CROSS SECTIONAL DRAWING WITH PUMP PARTS L...,NOT COMMENTED,PRJ-0000000017,P84,ATAPU 2 - P-84\02 - VENDOR DOCUMENTS\TOPSIDE\1...,shaikmeera.mohiddin,...,FINALIZADO,cesar.silvestre,17/12/2024 15:22:28,06/12/2024 04:22:41,CURRENT,KIB,https://integra-ext.petrobras.com.br/INTEGRA/#...,TECHNICAL ENGINEERING DOCUMENT,False,58
26901,I-DE-3010.2S-1251-311-SJA-003,Sim,B,1,PUMP PERFORMANCE CURVE FOR B-1251001A/B/C,COMMENTS ADDED,PRJ-0000000017,P84,ATAPU 2 - P-84\02 - VENDOR DOCUMENTS\TOPSIDE\1...,shaikmeera.mohiddin,...,FINALIZADO,cesar.silvestre,20/01/2025 18:27:00,09/01/2025 10:03:52,CURRENT,KIB,https://integra-ext.petrobras.com.br/INTEGRA/#...,TECHNICAL ENGINEERING DOCUMENT,False,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17795,I-PR-3010.2S-1251-974-SJA-202,Sim,0,1,PERFORMANCE AND STRING TEST PROCEDURE (FAT) FO...,None,PRJ-0000000017,P84,ATAPU 2 - P-84\02 - VENDOR DOCUMENTS\TOPSIDE\1...,shaikmeera.mohiddin,...,None,shaikmeera.mohiddin,16/01/2025 01:10:18,16/01/2025 01:08:03,CURRENT,KIB,https://integra-ext.petrobras.com.br/INTEGRA/#...,TECHNICAL ENGINEERING DOCUMENT,True,29
17792,I-PR-3010.2S-1251-974-SJA-203,Sim,B,1,WEIGHING PROCEDURE FOR SEA WATER BOOSTER PUMP ...,NOT COMMENTED,PRJ-0000000017,P84,ATAPU 2 - P-84\02 - VENDOR DOCUMENTS\TOPSIDE\1...,shaikmeera.mohiddin,...,FINALIZADO,cesar.silvestre,04/01/2025 11:18:09,27/12/2024 03:41:11,CURRENT,KIB,https://integra-ext.petrobras.com.br/INTEGRA/#...,TECHNICAL ENGINEERING DOCUMENT,False,43
15498,MA-3010.2S-1251-710-SJA-001,Sim,0,1,"MANUAL DE INSTALACAO, OPERACAO E MANUTENCAO DO...",COMMENTS ADDED,PRJ-0000000017,P84,ATAPU 2 - P-84\02 - VENDOR DOCUMENTS\TOPSIDE\1...,shaikmeera.mohiddin,...,FINALIZADO,imichelsen,11/12/2024 03:42:40,21/11/2024 09:17:41,CURRENT,KIB,https://integra-ext.petrobras.com.br/INTEGRA/#...,TECHNICAL ENGINEERING DOCUMENT,False,69
15497,MA-3010.2S-1251-710-SJA-101,Sim,0,1,"MANUAL DE INSTALACAO, OPERACAO E MANUTENCAO DO...",COMMENTS ADDED,PRJ-0000000017,P84,ATAPU 2 - P-84\02 - VENDOR DOCUMENTS\TOPSIDE\1...,shaikmeera.mohiddin,...,FINALIZADO,imichelsen,16/12/2024 22:17:56,28/11/2024 01:48:21,CURRENT,KIB,https://integra-ext.petrobras.com.br/INTEGRA/#...,TECHNICAL ENGINEERING DOCUMENT,False,64


#### Estratificação dos status dos documentos da Sulzer

In [41]:
# Lista de todos os status possíveis
status_list = ['APPROVED', 'RELEASED', 'COMMENTS ADDED', 'NOT RELEASED', 'NOT COMMENTED']

# Função para verificar se um valor é NaN ou None
def is_nan_or_none(x):
    return pd.isna(x) or x == '' or x.strip() == ''

# Realizar a contagem
status_counts = df_filtrado['release_status_vote'].value_counts(dropna=False)

# Criar um dicionário com as contagens, incluindo zeros para status não presentes
status_dict = {status: status_counts.get(status, 0) for status in status_list}

# Adicionar a contagem de NaN/None/Empty strings
nan_count = df_filtrado['release_status_vote'].apply(is_nan_or_none).sum()
status_dict['No Status (NaN/None/Empty)'] = nan_count

# Criar uma string formatada com os resultados
result_string = "\n".join([f"{status} = {count} documents" for status, count in status_dict.items()])

# Imprimir o resultado
print(result_string)


APPROVED = 0 documents
RELEASED = 1 documents
COMMENTS ADDED = 87 documents
NOT RELEASED = 1 documents
NOT COMMENTED = 13 documents
No Status (NaN/None/Empty) = 39 documents


#### Documentos atrasados em relação a LD da Sulzer

In [42]:
import pandas as pd
from datetime import datetime

rev_LD = 'PLANNED_DATE_2' # alterar para PLANNED_DATE_2, PLANNED_DATE_3 ou PLANNED_DATE_4 conforme revisáo da LD

# Converter as colunas de data para o tipo datetime
df_integra_last['emission_date'] = pd.to_datetime(df_integra_last['emission_date'], errors='coerce')
df_LD[rev_LD] = pd.to_datetime(df_LD[rev_LD], errors='coerce')

# Mesclar os dataframes com base no número do documento, usando df_LD como base
df_merged = pd.merge(df_LD, df_integra_last[['document', 'emission_date', 'title']],
      left_on='CLIENT_DOCUMENT', right_on='document', how='left')

# Função para determinar o status do documento
def get_document_status(row):
    if pd.isnull(row['emission_date']):
        if pd.notnull(row[rev_LD]) and row[rev_LD] < datetime.now():
            return 'Atrasado'
        else:
            return 'Pendente'
    else:
        return 'Emitido'

# Aplicar a função para determinar o status de cada documento
df_merged['status'] = df_merged.apply(get_document_status, axis=1)

# Calcular o atraso em dias para documentos atrasados
df_merged['dias_atraso'] = (datetime.now() - df_merged[rev_LD]).dt.days
df_merged.loc[df_merged['status'] != 'Atrasado', 'dias_atraso'] = 0

# Contagem total de documentos no df_LD
total_documentos = len(df_LD)
print(f"Total de documentos no df_LD: {total_documentos}")

# Exibir um resumo dos resultados
status_counts = df_merged['status'].value_counts()
print("Contagem de documentos por status:")
for status, count in status_counts.items():
    print(f"{status}: {count} ({count/total_documentos:.1%})")

# Exibir os documentos atrasados
documentos_atrasados = df_merged[df_merged['status'] == 'Atrasado']
print(f"\nDocumentos Atrasados ({len(documentos_atrasados)}):")
display(documentos_atrasados[['CLIENT_DOCUMENT', 'DOCUMENT_TITLE', rev_LD, 'dias_atraso']])

Total de documentos no df_LD: 316
Contagem de documentos por status:
Pendente: 175 (55.4%)
Emitido: 141 (44.6%)

Documentos Atrasados (0):


C:\Users\ELXY\AppData\Local\Temp\ipykernel_22928\2262938599.py:7: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_integra_last['emission_date'] = pd.to_datetime(df_integra_last['emission_date'], errors='coerce')


,CLIENT_DOCUMENT,DOCUMENT_TITLE,PLANNED_DATE_2,dias_atraso


#### Agrupa os meus pacotes contando o numero de linhas e trazendo a última data de assinatura da PO

In [43]:
# Filtrar as linhas que contêm os textos específico na coluna Package_Engineer_Petrobras.
df_pkg_cesar = df_escopo[df_escopo['Package_Engineer_Petrobras'].str.contains("Cesar Silvestre")]

# Realizar o agrupamento e conta as ocorrências
# df_grouped_pkg = df_pkg_cesar.groupby('PKG_DESCRIPTION').size().reset_index(name='Equip_counts')

# Realiza a contagem dos equipamentos para cada pacote usando o DataFrame df_equip caso contrario contagem no df_pkg_cesar
# Criar um dicionário a partir de df_equip para facilitar a busca 
equip_dict = dict(zip(df_equip['PKG_DESCRIPTION'], df_equip['QTY_EQUIPMENT']))

# Função para obter a contagem de equipamentos
def get_equip_count(pkg_description):
    if pkg_description in equip_dict:
        return equip_dict[pkg_description]
    else:
        return df_pkg_cesar[df_pkg_cesar['PKG_DESCRIPTION'] == pkg_description].shape[0]

# Realizar o agrupamento e contar as ocorrências
df_grouped_pkg = df_pkg_cesar.groupby('PKG_DESCRIPTION').apply(lambda x: pd.Series({
    'Equip_counts': get_equip_count(x.name)
})).reset_index()

# Adicionar a coluna Supplier com base no valor mais recente
df_grouped_pkg['Supplier'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_mapa_orbis.groupby('PKG_DESCRIPTION')['Supplier'].last() # Usa o método map para adicionar a coluna Supplier ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao último valor de Supplier correspondente no DataFrame original.
)

# Adicionar a coluna MR_Number com base no valor mais recente
df_grouped_pkg['MR_Number'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['MR_Number'].max() # Usa o método map para adicionar a coluna MR_Number ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de PO_Issue_Date_Plan correspondente no DataFrame original.
)

# Adicionar a coluna MR_Emitida com base no valor mais recente
df_grouped_pkg['MR_Emitida'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['MR_Emitida'].max() # Usa o método map para adicionar a coluna MR_Number ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de PO_Issue_Date_Plan correspondente no DataFrame original.
)

# Adicionar a coluna TBE_Number com base no valor mais recente
df_grouped_pkg['TBE_Number'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['TBE_Number'].max() # Usa o método map para adicionar a coluna MR_Number ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de PO_Issue_Date_Plan correspondente no DataFrame original.
)

# Adicionar a coluna TBE_Number com base no valor mais recente
df_grouped_pkg['TBE_Emitida'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['TBE_Emitida'].max() # Usa o método map para adicionar a coluna MR_Number ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de PO_Issue_Date_Plan correspondente no DataFrame original.
)

# Adicionar a coluna PO_Number com base no valor mais recente
df_grouped_pkg['PO_Number'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['PO_Number'].max() # Usa o método map para adicionar a coluna MR_Number ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de PO_Issue_Date_Plan correspondente no DataFrame original.
)

# Adicionar a coluna PO_Issue_Date_Plan com base no valor mais recente
df_grouped_pkg['PO_Issue_Date_Plan'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['PO_Issue_Date_Plan'].min() # Usa o método map para adicionar a coluna PO_Issue_Date_Plan ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de PO_Issue_Date_Plan correspondente no DataFrame original.
)

# Adicionar a coluna PO_Issue_Date_Replan com base no valor mais recente
df_grouped_pkg['PO_Issue_Date_Replan'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['PO_Issue_Date_Replan'].min() # Usa o método map para adicionar a coluna PO_Issue_Date_Plan ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de PO_Issue_Date_Plan correspondente no DataFrame original.
)

# Adicionar a coluna Critical com base no valor mais recente
df_grouped_pkg['Critical'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['Critical'].max() # Usa o método map para adicionar a coluna Critical ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de PO_Issue_Date_Plan correspondente no DataFrame original.
)

# Adicionar a coluna Required_On_Site_Date_Plan com base no valor mais recente
df_grouped_pkg['Required_On_Site_Date_Plan'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['Required_On_Site_Date_Plan'].max() # Usa o método map para adicionar a coluna Required_On_Site_Date_Plan ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de PO_Issue_Date_Plan correspondente no DataFrame original.
)

# Adicionar a coluna Pkg_ID 
df_grouped_pkg['Pkg_ID'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['Pkg_ID'].min() # Usa o método map para adicionar a coluna Pkg_ID ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de PO_Issue_Date_Plan correspondente no DataFrame original.
)

# Adicionar a coluna Pkg_Seatrium
df_grouped_pkg['Package_Seatrium'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['Package_Seatrium'].min() # Usa o método map para adicionar a coluna Pkg_Seatrium ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de Package_Seatrium correspondente no DataFrame original.
)

# Adicionar a coluna Pkg_Seatrium
df_grouped_pkg['Package_Expeditor'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['Package_Expeditor'].min() # Usa o método map para adicionar a coluna Pkg_Seatrium ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de Package_Seatrium correspondente no DataFrame original.
)

# Adicionar a coluna Pkg_Seatrium
df_grouped_pkg['Trigram'] = df_grouped_pkg['PKG_DESCRIPTION'].map(
    df_pkg_cesar.groupby('PKG_DESCRIPTION')['Trigram'].min() # Usa o método map para adicionar a coluna Pkg_Seatrium ao DataFrame df_grouped_pkg, mapeando cada PKG_DESCRIPTION ao valor máximo de Package_Seatrium correspondente no DataFrame original.
)


C:\Users\ELXY\AppData\Local\Temp\ipykernel_22928\831465176.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped_pkg = df_pkg_cesar.groupby('PKG_DESCRIPTION').apply(lambda x: pd.Series({


In [44]:
# Contando os documentos únicos na coluna CLIENT_DOCUMENT 
conta_docs_sulzer = df_VDA['CLIENT_DOCUMENT'].value_counts()
# Contando os documentos únicos na coluna "document"
conta_docs_emitidos_sulzer = df_filtrado['document'].value_counts()

# Adicionando a contagem de documentos na célula correspondente a 'WATER INJECTION PUMPS'  
df_grouped_pkg.loc[df_grouped_pkg['PKG_DESCRIPTION'] == 'WATER INJECTION PUMPS', 'n_docs'] = conta_docs_sulzer.sum()
# Adicionando a contagem de documentos na célula correspondente a 'WATER INJECTION PUMPS'  
df_grouped_pkg.loc[df_grouped_pkg['PKG_DESCRIPTION'] == 'WATER INJECTION PUMPS', 'n_docs_emitidos'] = conta_docs_emitidos_sulzer.sum()

print("Valores Agrupados - DataFrame df_grouped_pkg:")
display(df_grouped_pkg)


Valores Agrupados - DataFrame df_grouped_pkg:


,PKG_DESCRIPTION,Equip_counts,Supplier,MR_Number,MR_Emitida,TBE_Number,TBE_Emitida,PO_Number,PO_Issue_Date_Plan,PO_Issue_Date_Replan,Critical,Required_On_Site_Date_Plan,Pkg_ID,Package_Seatrium,Package_Expeditor,Trigram,n_docs,n_docs_emitidos
0,API 610 CENTRIFUGAL PUMPS,90,KSB BRASIL LTDA,I-RM-3010.2S-1200-311-S2N-002,Sim,I-PT-3010.2S-1200-311-S2N-002,NaN,CON-P85-SNT-PROC-0002-25,2025-02-01,NaT,NaN,2026-06-08,P85-Top-014,,NaN,NaN,NaN,NaN
1,DECK TROLLEY,6,None,I-RM-3010.2S-5266-620-S2N-001,NaN,I-PT-3010.2S-5266-620-S2N-001,NaN,NaN,2025-07-14,NaT,NaN,2026-12-31,P85-Top-034,,NaN,NaN,NaN,NaN
2,FLARE SYSTEM,7,None,I-RM-3010.2S-5412-583-S2N-001,NaN,I-PT-3010.2S-5412-583-S2N-001,NaN,NaN,2025-05-17,2025-07-28,Critical,2026-08-31,P85-Top-028,Lim Kwang Heng,NaN,NaN,NaN,NaN
3,FRESH WATER CHLORINATION UNIT,2,DE NORA DO BRASIL LTDA,I-RM-3010.2S-5122-560-S2N-042,Sim,I-PT-3010.2S-5122-560-S2N-042,Sim,CON-P85-SNT-PROC-0006-24,2025-03-31,2025-01-31,NaN,2026-05-30,P85-Top-040,Cecilia Loong Siew Lee,NaN,D5A,NaN,NaN
4,NITROGEN GENERATOR UNITS,152,GARDNER DENVER KOREA,I-RM-3010.2S-5241-470-S2N-001,Sim,I-PT-3010.2S-5241-470-S2N-001,Sim,CON-P85-SSG-PROC-0001-25,2024-12-17,2025-02-15,NaN,2026-04-30,P85-Top-038,Cecilia Loong Siew Lee,Natalia Motosi,GK1,NaN,NaN
5,NON-API 610 CENTRIFUGAL PUMPS,17,None,I-RM-3010.2S-1200-311-S2N-001,Sim,I-PT-3010.2S-1200-311-S2N-001,NaN,NaN,2025-02-24,2025-03-07,NaN,2026-06-08,P85-Top-015,Cecilia Loong Siew Lee,NaN,NaN,NaN,NaN
6,PIG LAUNCHERS AND RECEIVERS,27,None,I-RM-3010.2S-1210-296-S2N-001,Sim,I-PT-3010.2S-1210-296-S2N-001,NaN,NaN,2025-03-07,NaT,NaN,2026-07-30,P85-Top-035,,NaN,NaN,NaN,NaN
7,PROGRESSIVE CAVITY PUMPS,8,None,I-RM-3010.2S-1200-313-S2N-001,Sim,I-PT-3010.2S-1200-313-S2N-001,NaN,NaN,2025-02-27,NaT,NaN,2026-07-15,P85-Top-017,Muhamad Aiman Arif bin Mohd,NaN,NaN,NaN,NaN
8,RECIPROCATING PUMPS,15,None,I-RM-3010.2S-1200-312-S2N-001,NaN,I-PT-3010.2S-5133-312-S2N-001,NaN,NaN,2025-04-29,NaT,Critical,2026-06-08,P85-Top-016,Muhamad Aiman Arif bin Mohd,NaN,NaN,NaN,NaN
9,SEA WATER ELECTROCHLORINATION UNIT,2,DE NORA DO BRASIL LTDA,I-RM-3010.2S-5121-560-S2N-040,Sim,I-PT-3010.2S-5121-560-S2N-040,Sim,CON-P85-SNT-PROC-0006-24,2025-03-31,NaT,NaN,2026-05-30,P85-Top-039,Cecilia Loong Siew Lee,NaN,D5A,NaN,NaN


#### Verifica a VDA emitida deve ser YES ou NO no WF de comentários

In [45]:
# Verifica a LD da Sulzer e todos os arquivos do Integra

# Cria uma função para adicionar o padrão regex:
def criar_regex_pattern(*textos_busca): 
    return ''.join(f"(?=.*{texto})" for texto in textos_busca if texto) 
 
texto_busca1 = "VDA"
texto_busca2 = "SJA" 
texto_busca3 = ""  # Por exemplo posso deixar o terceiro texto de busca em branco 
texto_busca4 = ""
 
# Criando a expressão regular 
regex_pattern = criar_regex_pattern(texto_busca1, texto_busca2, texto_busca3, texto_busca4) 
 
# Filtrando o DataFrame 
df_filtrado_VDA = df_integra_last[df_integra_last['document'].str.contains(regex_pattern, regex=True, na=False)]

# Função para extrair a parte inicial do texto
def extract_initial_text(text):
    return text.split(',')[0]

# Aplicando a função para extrair a parte inicial do texto 
df_filtrado_VDA['reference_document'] = df_filtrado_VDA['reference_document'].apply(extract_initial_text) 

# Fazendo o merge para incluir a coluna Have_a_comment no df_filtrado_VDA 
df_filtrado_VDA = df_filtrado_VDA.merge(df_VDA[['CLIENT_DOCUMENT', 'Have_a_comment']], left_on='reference_document', right_on='CLIENT_DOCUMENT', how='left') 

# Listando apenas as colunas desejadas 
colunas_desejadas = ['document', 'title', 'reference_document', 'CLIENT_DOCUMENT', 'Have_a_comment', 'atrasado', 'dias_uteis'] 
df_selecionado = df_filtrado_VDA[colunas_desejadas]

df_selecionado

C:\Users\ELXY\AppData\Local\Temp\ipykernel_22928\1909907603.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado_VDA['reference_document'] = df_filtrado_VDA['reference_document'].apply(extract_initial_text)


,document,title,reference_document,CLIENT_DOCUMENT,Have_a_comment,atrasado,dias_uteis
0,VDA-P84-SJA-00001,GENERAL ARRANGEMENT DRAWING FOR B-1251002A/B/C,I-DE-3010.2S-1251-311-SJA-101,I-DE-3010.2S-1251-311-SJA-101,YES,False,79
1,VDA-P84-SJA-00002,GENERAL ARRANGEMENT DRAWING FOR B-1251001A/B/C,I-DE-3010.2S-1251-311-SJA-001,I-DE-3010.2S-1251-311-SJA-001,YES,False,79
2,VDA-P84-SJA-00003,RECOMMENDED SPARE PARTS LIST FOR TWO YEARS OPE...,I-LI-3010.2S-1251-970-SJA-002,I-LI-3010.2S-1251-970-SJA-002,YES,False,42
3,VDA-P84-SJA-00004,UTILITY CONSUMPTION LIST FOR B-1251003A/B,I-LI-3010.2S-1251-940-SJA-202,I-LI-3010.2S-1251-940-SJA-202,YES,True,28
4,VDA-P84-SJA-00005,LIST OF VENDORS DOCUMENTS AND DRAWINGS FOR ALL...,I-LD-3010.2S-1251-911-SJA-001,I-LD-3010.2S-1251-911-SJA-001,YES,False,32
...,...,...,...,...,...,...,...
112,VDA-P84-SJA-00113,MECHANICAL SEAL DRAWING (WITH MATERIAL LIST) F...,I-DE-3010.2S-1251-393-SJA-201,I-DE-3010.2S-1251-393-SJA-201,YES,True,28
113,VDA-P84-SJA-00114,HYDROSTATIC TEST PROCEDURE MAIN INJECTION WATE...,I-PR-3010.2S-1251-974-SJA-101,I-PR-3010.2S-1251-974-SJA-101,YES,True,26
114,VDA-P84-SJA-00115,I/O LIST FOR B-1251001A/B/C,I-LI-3010.2S-1251-940-SJA-009,I-LI-3010.2S-1251-940-SJA-009,NaN,False,26
115,VDA-P84-SJA-00116,SCHEMATIC / FUNCTIONAL DIAGRAMS FOR ELECTRICAL...,I-DE-3010.2S-1251-946-SJA-105,I-DE-3010.2S-1251-946-SJA-105,YES,True,26


#### Salva os dados em Arquivo Excel

In [46]:
control_cesar = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\2025-02-26_P84-P85_controle_cesar.xlsx'

# Salvando arquivos P84 e P85:
with pd.ExcelWriter(control_cesar) as writer:
    df_integra.to_excel(writer, sheet_name='integra_aida_all_docs', index=False)
    df_filtrado.to_excel(writer, sheet_name='df_sulzer', index=False)
    df_escopo.to_excel(writer, sheet_name='P84 Topside Map', index=False)
    df_grouped_pkg.to_excel(writer, sheet_name='Resumo', index=False)
    df_wf.to_excel(writer, sheet_name='wf', index=False)
    df_filtrado_VDA.to_excel(writer, sheet_name='have_a_comment', index=False)
print(f'Arquivo {control_cesar} salvo em: ', Path.cwd())

Arquivo C:\Users\elxy\Documents\Codigos\Python\P84_85\2025-02-26_P84-P85_controle_cesar.xlsx salvo em:  c:\Users\ELXY\Documents\Codigos\Python\P84_85
